In [61]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from scipy.optimize import minimize
from scipy.stats import t, norm
from scipy.stats import ttest_1samp
from scipy.stats import bartlett, levene
from scipy.stats import kurtosistest, skewtest
from scipy.stats import chisquare
import scipy.stats as stats
from scipy.stats import multivariate_normal
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [76]:
data = pd.read_csv('Funding Rate_ETHUSDTPerpetual.csv')

# Convert 'Funding Rate' from string to float and remove the '%' symbol
data['Funding Rate'] = data['Funding Rate'].str.rstrip('%').astype('float') / 100

# Convert 'Time' to datetime format and extract the date for grouping
data['Time'] = pd.to_datetime(data['Time'])
data['date'] = data['Time'].dt.date

# Group by the new 'Date' column and calculate the mean 'Funding Rate' for each date
daily_avg_funding_rate = data.groupby('date')['Funding Rate'].mean().reset_index()
# daily_avg_funding_rate.set_index('date', inplace=True)
# Show the calculated daily average funding rates
daily_avg_funding_rate

df = pd.read_csv('merged_btc_usdt.csv')
df = df.drop('Funding Rate', axis=1)
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df['margin_lending_rate_t-1'] = df['stable_borrow_apy'].shift(1)
df['daily_return'] = df['price'].pct_change()
df.rename(columns={'BitVol® Bitcoin Volatility Index':'BTC_implied_volatility','daily_return':'R_BTC'}, inplace=True)
combined_df = pd.merge(daily_avg_funding_rate, df, on='date', how='inner', sort=True)
combined_df.head(10)

,date,Funding Rate,BTC_implied_volatility,stable_borrow_apy,price,market_cap,total_volume,margin_lending_rate_t-1,R_BTC
0,2020-12-02,0.000219,91.31,0.042101,0.994794,19258843705,6.352755e+10,NaN,NaN
1,2020-12-03,0.000415,89.73,0.080037,0.999091,19392984903,4.788921e+10,0.042101,0.004319
2,2020-12-04,0.000176,86.98,0.105821,1.001262,19453232188,3.984176e+10,0.080037,0.002173
3,2020-12-05,0.000120,85.23,0.068982,1.003987,19616345385,4.371844e+10,0.105821,0.002722
4,2020-12-06,0.000100,86.25,0.139586,1.001683,19673710812,3.902534e+10,0.068982,-0.002294
5,2020-12-07,0.000165,86.94,0.290737,1.001337,19692101877,3.494903e+10,0.139586,-0.000346
6,2020-12-08,0.000139,84.71,0.212760,1.000365,19695202345,3.475867e+10,0.290737,-0.000970
7,2020-12-09,0.000100,79.24,0.078127,0.998170,19756413071,3.893177e+10,0.212760,-0.002194
8,2020-12-10,0.000125,78.25,0.092904,1.000598,19731593806,4.847821e+10,0.078127,0.002432
9,2020-12-11,0.000108,76.50,0.067873,1.000863,19775400468,3.686815e+10,0.092904,0.000265


In [73]:
# model 1
x = combined_df['Funding Rate']
y = combined_df['stable_borrow_apy']

# add a constant
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      stable_borrow_apy   R-squared:                       0.319
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     539.3
Date:                Tue, 09 Apr 2024   Prob (F-statistic):           4.03e-98
Time:                        13:35:09   Log-Likelihood:                 1842.2
No. Observations:                1151   AIC:                            -3680.
Df Residuals:                    1149   BIC:                            -3670.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.1283      0.002     80.041   

In [85]:
# model 2
# x = combined_df['Funding Rate']
# y = combined_df['stable_borrow_apy']

combined_df['date'] = pd.to_datetime(combined_df['date'])

combined_df['month'] = combined_df['date'].dt.month
month_dummies = pd.get_dummies(combined_df['month'], prefix='month', drop_first=True)
combined_df = pd.concat([combined_df, month_dummies], axis=1)

x = combined_df['Funding Rate']
y = combined_df['stable_borrow_apy']
x = sm.add_constant(x)


month_fe_formula = ' + '.join(month_dummies.columns)

formula = 'stable_borrow_apy ~ Q("Funding Rate") + ' + month_fe_formula

model = smf.ols(formula, data=combined_df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      stable_borrow_apy   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.330
Method:                 Least Squares   F-statistic:                     48.27
Date:                Tue, 09 Apr 2024   Prob (F-statistic):           4.74e-93
Time:                        13:44:36   Log-Likelihood:                 1857.5
No. Observations:                1151   AIC:                            -3689.
Df Residuals:                    1138   BIC:                            -3623.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1336      0.00

In [86]:
# model 3
combined_df['date'] = pd.to_datetime(combined_df['date'])

combined_df['month'] = combined_df['date'].dt.month
month_dummies = pd.get_dummies(combined_df['month'], prefix='month', drop_first=True)
combined_df = pd.concat([combined_df, month_dummies], axis=1)

x = combined_df[['Funding Rate', 'margin_lending_rate_t-1','BTC_implied_volatility', 'R_BTC']]
y = combined_df['stable_borrow_apy']
x = sm.add_constant(x)

month_fe_formula = ' + '.join(month_dummies.columns)

formula = 'stable_borrow_apy ~ Q("Funding Rate") + ' + month_fe_formula

model = smf.ols(formula, data=combined_df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      stable_borrow_apy   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.330
Method:                 Least Squares   F-statistic:                     48.12
Date:                Tue, 09 Apr 2024   Prob (F-statistic):           8.60e-93
Time:                        13:44:55   Log-Likelihood:                 1856.9
No. Observations:                1151   AIC:                            -3688.
Df Residuals:                    1138   BIC:                            -3622.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1341      0.00